In [14]:
import gymnasium as gym
import highway_env
from utils import record_videos, show_videos
import pprint
import matplotlib.pyplot as plt
import numpy as np
import os
import agent
os.environ["CUDA_VISIBLE_DEVICES"]="-1"


In [25]:
env = gym.make('racetrack-v0')
# env.close()
env.configure({
    'action': {'lateral': True,
            'longitudinal': False,
            'target_speeds': [0, 5],
            'type': 'ContinuousAction'},
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 1,
        "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h", 
                     "heading", "long_off", "lat_off", "ang_off"],
    },
    "other_vehicles": 1,
    'show_trajectories': True,
     'offroad_terminal': True,
})
# pprint.pprint(env.config)
(obs, info), done = env.reset(), False
print(obs.flatten())
print(type(obs))
print("Environment is setted up.")



[ 1.0000000e+00  3.1210881e-01  6.2500000e-01  1.2500000e-01
  0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
  2.0421766e+01  0.0000000e+00  0.0000000e+00  1.0000000e+00
  2.7368924e-01 -8.5237885e-01 -6.0356297e-02 -6.1012201e-02
  7.2723877e-01 -6.8638456e-01 -7.5650585e-01  1.8912647e+01
  3.5527137e-15  0.0000000e+00]
<class 'numpy.ndarray'>
Environment is setted up.


/home/tinrafiq/.local/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/home/tinrafiq/.local/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -inf, upper bound: inf. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/home/tinrafiq/.local/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [16]:
agent = agent.DDQNAgent(alpha = 0.001, gamma = 0.999, epsilon = 1.0, 
                       obs_shape = obs.shape, batch_size = 64, epsilon_dec = 0.993, 
                       epsilon_end = 0.05, mem_size = 100000, min_mem_size = 100, 
                       replace_target = 1000, learning_rate=0.001)
print("Agent is initialized.")


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 22)                0         
                                                                 
 dense_12 (Dense)            (None, 512)               11776     
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                                 
 dense_14 (Dense)            (None, 9)                 4617      
                                                                 
Total params: 279,049
Trainable params: 279,049
Non-trainable params: 0
_________________________________________________________________
Agent is initialized.


In [17]:
best_score = -1000.0
score_history = []

In [20]:
try:
    env = record_videos(env)

    for episode in range(2000):
        (observation, info), done = env.reset(), False
        done_ = False
        score = 0
        step = 0

        while True:
            action, action_index = agent.get_action(observation)
            new_observation, reward, done, truncated, info = env.step(action=[action])

            print("observation shape : ", observation.shape)
            print("new obs shape : ", new_observation.shape)
            if info["crashed"] == True or info["rewards"]["on_road_reward"] == False:
                done_ = True
                reward = -1.0
            else: done_ = False

            score +=reward
            agent.remember(state=observation, action=action_index, done=done,
                           reward=reward, new_state=new_observation)
            agent.train()

            observation = new_observation

            if done or done_:
                break

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score:
            best_score = avg_score
            agent.save_model()

        print('episode ', episode, 'score %.1f' % score,
               'avg score %.1f' % avg_score)

except KeyboardInterrupt:
    print("Simulator Interrupted")

env.close()

/home/tinrafiq/.local/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:87: UserWarning: WARN: Overwriting existing videos at /home/tinrafiq/Documents/thesis/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/home/tinrafiq/.local/lib/python3.8/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:62: UserWarning: WARN: Disabling video recorder because environment <RecordVideo<RecordVideo<OrderEnforcing<PassiveEnvChecker<RacetrackEnv<racetrack-v0>>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(


1/1 [==============================] - 2s 2s/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 30ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 43ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 26ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 27ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 26ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 24ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 23ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 24ms/step
observation shape :  (2, 11)
new obs shape :  (2, 11)
1/1 [==============================] - 0s 24ms/s